## 1. 필요 라이브러리 선언

In [8]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("./"))))
from common import commonFunc as cf
from datetime import datetime, timedelta
import time
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

## 2. 필요 변수 선언

### 2-1. 메타데이터 업로드

In [9]:
metadata = pd.read_excel("../../input/datalake_meta22.xlsx", sheet_name="3. 통계청")

### 2-2. output 폴더 생성 변수

In [10]:
SITENAME = "kostat"
DATANAME= "기상관측일자료목록조회"
targetData = metadata.loc[metadata.자료명==DATANAME]
SERVICENAME = targetData["서비스키"].values[0]

### 2-3. 기본키 설정

In [11]:
stnIdsDf = pd.read_csv("./기상관측지점.csv", encoding="ms949")
serviceKey = "h2pHFUSHMnsx/wJfhd+fyC4L0X+g16L0FhJvDcos8Px4Fqttih1HONUkKXQ5ITmYlq1vjIYT8/G+twwMfA9m8Q=="
startDt = "20120101"
endDt = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
numOfRows = "999"
dataType = "JSON"
dataCd = "ASOS"
dateCd = "DAY"
stnIdsLst = stnIdsDf.지점.values.tolist()

BASEPARAM_KEY = targetData.기본키.values[0].split(",")
BASEPARAM_Lst = []

BASEPARAM_VAL = [serviceKey, numOfRows, dataCd, dateCd, startDt, endDt]

for i in range(len(stnIdsLst)):
    BASEPARAM = {}
    BASEPARAM_VAL.append(stnIdsLst[i])
    for j in range(len(BASEPARAM_KEY)):
        BASEPARAM[BASEPARAM_KEY[j]] = BASEPARAM_VAL[j]
        BASEPARAM["dataType"] = dataType
    BASEPARAM_Lst.append(BASEPARAM)
    BASEPARAM_VAL.pop()

### 2-4. 함수 파라미터

In [12]:
URL = targetData["URL"].values[0]
PAGEYN=1
STDENCODING='utf-8'
inType = "jsonnormal"
mode = 1

## 3. 데이터 수집

### 3-1. 기존 수집 데이터 존재 여부 체크

In [13]:
OUTPUTPATH="../../output"
filePath = os.path.join(OUTPUTPATH,SITENAME,DATANAME,SERVICENAME) + '.csv'
if os.path.isfile(filePath):
    os.remove(filePath)

### 3-2. 데이터 수집

In [14]:
breakPoint = 0

starttime = time.time()
print("수집시작 : ", time.strftime('%c', time.localtime(time.time())))

for i in range(len(BASEPARAM_Lst)):
    try:
        scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM_Lst[i],PAGEYN,jsonkey="item",inType=inType)
        resultDf = scrapyResult[0]
        print(resultDf)
        cf.savedata(resultDf,SITENAME,DATANAME,SERVICENAME,mode=mode)
    except Exception as e:
        print(e)
        breakPoint = i
        if e.args[0] == "22":
            print("LIMITED_NUMBER_OF_SERVICE_REQUESTS_EXCEEDS_ERROR")
        elif e.args[0] == "01" or e.args[0] == "04":
            try:
                time.sleep(5)
                scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM_Lst[i],PAGEYN)
                resultDf = scrapyResult[0]
                print(resultDf)
                cf.savedata(resultDf,SITENAME,DATANAME,SERVICENAME,mode=mode)
                continue
            except Exception as e:
                print(e)
                continue
        break

print("종료 시점 파라미터키 : ",BASEPARAM_Lst[breakPoint])
print("수집 종료 : ", time.time()-starttime)

수집시작 :  Thu Apr 28 10:48:12 2022
1 page scraping start
2 page scraping start
3 page scraping start
4 page scraping start
5 page scraping start
03
dataframe기상관측일자료목록조회, param:{'serviceKey': 'h2pHFUSHMnsx/wJfhd+fyC4L0X+g16L0FhJvDcos8Px4Fqttih1HONUkKXQ5ITmYlq1vjIYT8/G+twwMfA9m8Q==', 'dataType': 'JSON', 'numOfRows': '999', 'dataCd': 'ASOS', 'dateCd': 'DAY', 'startDt': '20120101', 'endDt': '20220427', 'stnIds': 90, 'pageNo': 5} rows: 62 completed
    stnId stnNm          tm avgTa minTa minTaHrmt maxTa maxTaHrmt mi10MaxRn  \
0      90    속초  2012-01-01   1.1  -1.5       719   4.9      1031             
1      90    속초  2012-01-02  -0.3  -2.9       827   4.0      1236             
2      90    속초  2012-01-03  -0.8  -4.2       357   3.7      1237             
3      90    속초  2012-01-04  -2.6  -5.6       517   1.8      1451             
4      90    속초  2012-01-05  -2.3  -5.8       429   2.8      1357             
..    ...   ...         ...   ...   ...       ...   ...       ...       ...   
7

KeyboardInterrupt: 